<p align="center">
  <img src="https://user-images.githubusercontent.com/90031508/183531098-494a5819-7714-4f72-8ff8-d038982eb5f0.png" alt="Water Oracle logo"/>
</p>



# Image export

This Work is adapted from 'Tensorflow example workflows', 
https://developers.google.com/earth-engine/guides/tf_examples examples.
Copyright 2020 Google LLC. https://www.apache.org/licenses/LICENSE-2.0.

Please run this notebook on google colab (pro+)

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/drive/1kPxIHJQVvns-vKqs0QuYJIZpFRMR78gz?usp=sharing">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/ese-msc-2021/irp-kl121"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Introduction


## Prerequisites
- Google account and logins
- Google colab subscription with pro or pro+ is optional but would help with long runtime
- Google cloud platform account in order to use google cloud bucket. (Note that you would need sufficient funds to store large amount of models and training data.)
- Wandb.ai account which is free of charge

## What is this notebook?

The main purpose of this notebook is to export image in regions of interest and predict this image and generate the output in as google cloud asset in google earth engine.

Initially, the required data is imported from the earth engine repository.

- The data used includes:
Sentinel-1 Data (10m) 
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD?hl=en

- USG’s Landsat-8 Collection 1 and Tier 1 (30m) and the cloud is masked
https://developers.google.com/earth-engine/guides/landsat
- NASADEM: NASA NASADEM Digital Elevation (30m) https://developers.google.com/earth-engine/datasets/catalog/NASA_NASADEM_HGT_001

<b>The notebook is splitted into three main objectives:</b>
1. Enable user to select any of the models written in the paper to predict the water bodies and export this image

2. Case study of the NASADEM data and how it can improve of affect when pairing with Landsat-8 or sentinel-1

3. Apply the GlobalWaterNet in order to understand the seasonal effect of water bodies in Tibet

4. Apply THWaterNet in order to map southern Thailand Flooding extent in 2016

5. GlobalWaterNet to predict 10 different water bodies in different countries

## Creating Packages

Creating the tools packages that will be used throughout the notebook. The package includes
- metrics_.py
- config.py
- preprocessing.py
- losses_.py
- images.py

In [ ]:
PACKAGE_PATH = 'tools'

!ls -l
!mkdir {PACKAGE_PATH}
!touch {PACKAGE_PATH}/__init__.py
!ls -l {PACKAGE_PATH}

total 4
drwxr-xr-x 1 root root 4096 Aug  3 20:21 sample_data
total 0
-rw-r--r-- 1 root root 0 Aug  7 23:21 __init__.py


In [ ]:
%%writefile {PACKAGE_PATH}/metrics_.py

from keras import backend as K
import tqdm.notebook as tq
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

CONFIG = None

__all__ = ["f1", "custom_accuracy", "MetricCalculator", "MetricCalculator_multiview_2", "MetricCalculator_multiview_3", "MetricCalculator_NDWI", "ndwi_threashold"]

def f1(y_true, y_pred):
    """
    The function is used as tensorflow metrics when training. It takes in the ground truth and the
    model predicted result and evaluate the F1 score. This is an experimental function and should not be used as
    further model training metric.

    Parameters
    ----------
    y_true : tf.tensor
    y_pred : tf.tensor

    Returns
    ----------
    F1 score in keras backend

    Notes
    -----
    This function is flawed because keras calculates the metrics batchwise 
    which is why F1 metric is removed from keras. To properly calulate the F1 score, we can use the callback function
    or manually calculate F1 score after the model has finished training. The latter is chosen and this could be seen
    in MetricCalculator, MetricCalculator_multiview_2 and MetricCalculator_multiview_3.
  
    The reason this function is kept is because the model was initially trained with these metrics and
    stored in the google cloud bucket. To retrieve the models these metrics must be passed inorder to retrieve the model.
    Since the model is optimize on the loss rather than the metrics, the incorrect metric would not effect the model
    training process. The code is obtained/modified from:

    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

    https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
    """
    def recall(y_true, y_pred):
        """
        Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """
        Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def custom_accuracy(y_true, y_pred):
    """
    The function is used as tensorflow metrics when training. It takes in the ground truth and the
    model predicted result and evaluate the accuracy score. This is an experimental function and should not be used as
    further model training metric.

    Parameters
    ----------
    y_true : tf.tensor
    y_pred : tf.tensor

    Returns
    ----------
    accuracy score in keras backend

    Notes
    -----
    This function is modified from the F1 metric above to fit the definition of accuracy. However, tensorflow's
    "categorical_accuracy" is used instead. The accuracy metric would also be recalculated again in 
    MetricCalculator, MetricCalculator_multiview_2 and MetricCalculator_multiview_3.
  
    The reason this function is kept is because the model was initially trained with these metrics and
    stored in the google cloud bucket. To retrieve the models these metrics must be passed inorder to retrieve the model.
    Since the model is optimize on the loss rather than the metrics, the incorrect metric would not effect the model
    training process. The code is obtained/modified from:

    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

    https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
    """
    # total_data = K.int_shape(y_true) + K.int_shape(y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    true_negatives = K.sum(K.round(K.clip(1 - y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    total_data = - true_positives + true_negatives + possible_positives + predicted_positives
    return (true_positives + true_negatives) / (total_data + K.epsilon())



def MetricCalculator(model, test_data, total_steps):
  """
  This function takes in the feature stack model loaded from google cloud bucket, the test_data which is the tensor object and
  the number of steps and returns the metrics including accuracy, recall, precision and F1

  Parameters
  ----------
  model : keras.engine.functional.Functional
  test_data : RepeatDataset with tf.float32
  total_steps : int/float

  Returns
  ----------
  Returns the precision, recall, f1, accuracy metric based on the model performance.

  Notes
  -----
  This function should be used instead of the F1, custom_accuracy written above. The code is obtained/modified from:

  https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

  https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
  """
  pred = []
  true = []
  pbar = tq.tqdm(total=total_steps)
  for steps, data in enumerate(test_data):
    # print(f'Number of steps: {steps}', end = "\r")
    pbar.update(1)
    if steps == total_steps:
      break
    input = data[0]
    y_true = data[1]
    y_pred = np.rint(model.predict(input))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)


  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy



def MetricCalculator_multiview_2(model, test_data, total_steps):
  """
  This function takes in the multiview-2 model loaded from google cloud bucket, the test_data which is the tensor object and
  the number of steps and returns the metrics including accuracy, recall, precision and F1

  Parameters
  ----------
  model : keras.engine.functional.Functional
  test_data : RepeatDataset with tf.float32
  total_steps : int/float

  Returns
  ----------
  Returns the precision, recall, f1, accuracy metric based on the model performance.

  Notes
  -----
  This function should be used instead of the F1, custom_accuracy written above. The code is obtained/modified from:

  https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

  https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
  """
  pbar = tq.tqdm(total=total_steps)
  pred = []
  true = []
  for steps, data in enumerate(test_data):
    pbar.update(1)
    if steps >= total_steps:
      break
    input = data[0]
    x1, x2 = tf.split(input, [len(CONFIG.BANDS1),len(CONFIG.BANDS2)], 3)
    y_true = data[1]
    y_pred = np.rint(model.predict([x1, x2]))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)
  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy

def MetricCalculator_multiview_3(model, test_data, total_steps):
  """
  This function takes in the multiview-3 model loaded from google cloud bucket, the test_data which is the tensor object and
  the number of steps and returns the metrics including accuracy, recall, precision and F1

  Parameters
  ----------
  model : keras.engine.functional.Functional
  test_data : RepeatDataset with tf.float32
  total_steps : int/float

  Returns
  ----------
  Returns the precision, recall, f1, accuracy metric based on the model performance.

  Notes
  -----
  This function should be used instead of the F1, custom_accuracy written above. The code is obtained/modified from:

  https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

  https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
  """
  pbar = tq.tqdm(total=total_steps)
  pred = []
  true = []
  for steps, data in enumerate(test_data):
    pbar.update(1)
    if steps >= total_steps:
      break
    input = data[0]
    x1, x2, x3 = tf.split(input, [len(CONFIG.BANDS1),len(CONFIG.BANDS2),len(CONFIG.BANDS3)], 3)
    y_true = data[1]
    y_pred = np.rint(model.predict([x1, x2, x3]))
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)
  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy


def ndwi_threashold(B3, B5):
  """
  This function takes in bands 3 and bands 5 from the landsat imagery and returns the tuple prediction of
  whether there is water present or not. The threashold is set at 0.

  Parameters
  ----------
  test_data : RepeatDataset with tf.float32
  total_steps : int/float

  Returns
  ----------
  tuple of whether there is water or not
  """
  ndwi = (B3-B5)/(B3+B5)
  if ndwi > 0:
    return 0, 1
  else:
    return 1, 0

def MetricCalculator_NDWI(test_data, total_steps):
  """
  This function takes in the test_data which is the tensor object and
  the number of steps and returns the metrics including accuracy, recall, precision and F1
  for NDWI performance.

  Parameters
  ----------
  test_data : RepeatDataset with tf.float32
  total_steps : int/float

  Returns
  ----------
  Returns the precision, recall, f1, accuracy metric based on the NDWI performance
  """
  pred = []
  true = []
  pbar = tq.tqdm(total=total_steps)
  for steps, data in enumerate(test_data):
    # print(f'Number of steps: {steps}', end = "\r")
    pbar.update(1)
    if steps == total_steps:
      break
    input = data[0]
    y_true = data[1]
    input = np.reshape(input, (256*256,2))
    y_pred = []
    for i in range(256*256):
      B3, B5 = input[i]
      first, second = ndwi_threashold(B3, B5)
      y_pred.append([first, second])
    y_true = np.reshape(y_true, (256*256,2))
    y_pred = np.reshape(y_pred, (256*256,2))
    pred.append(y_pred)
    true.append(y_true)


  f1_macro = f1_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  recall_macro= recall_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  precision_macro = precision_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)), average="macro")
  accuracy = accuracy_score(np.reshape(true, (total_steps*65536, 2)), np.reshape(pred, (total_steps*65536, 2)))

  print("precision_macro: ", precision_macro)
  print("recall_macro: ", recall_macro)
  print("F1_macro_Score: : ", f1_macro)
  print("Accuracy: ", accuracy)

  return precision_macro, recall_macro, f1_macro, accuracy

In [ ]:
%%writefile {PACKAGE_PATH}/config.py

import tensorflow as tf
from . import metrics_

__all__ = ["configuration"]

class configuration:
  """
  In each experiment, the combinations of satellite's bands that is used to train the neural network is different.
  Also the way to train the neural network is also different, whether it is feature stack, multiview learning with two
  or three perceptrons. As each experiment has different settings, it is important to store them and reuse this
  throughout the project. This class enables user to store the settings and reuse the settings.
  """
  def __init__(self, PROJECT_TITLE, BANDS1, TRAIN_SIZE, EVAL_SIZE, BANDS2=[], BANDS3=[], country="TH", image=None, sam_arr=None, type_=1, LOSS="categorical_crossentropy", EPOCHS=10, BATCH_SIZE = 16, dropout_prob=0.3):
    """

    Initialising/storing the parameters to use later

    Parameters
    ----------
    PROJECT_TITLE : string
    BANDS1 : list
    TRAIN_SIZE : int/float
    EVAL_SIZE : int/float
    BANDS2 : list
    BANDS3 : list
    country : string
    image : ee.image.Image
    sam_arr : ee.image.Image
    type : int/float

    """
    if type_ == 1:
      self.type_ = "fs"
    elif type_ == 2:
      self.type_ = "m2"
    elif type_ == 3:
      self.type_ = "m3"
    else:
      self.type_ = None
    self.country = country
    self.PROJECT_TITLE = PROJECT_TITLE
    self.BANDS1 = BANDS1
    self.BANDS2 = BANDS2
    self.BANDS3 = BANDS3
    self.BUCKET = "geebucketwater"
    self.FOLDER = f'{self.type_}_{self.country}_Cnn_{self.PROJECT_TITLE}'
    self.TRAIN_SIZE = TRAIN_SIZE
    self.EVAL_SIZE = EVAL_SIZE
    self.BUCKET = "geebucketwater"
    self.TRAINING_BASE = f'training_patches'
    self.EVAL_BASE = f'eval_patches'
    self.TEST_BASE = f'test_patches'
    self.RESPONSE = 'water'
    self.BANDS = BANDS1 + BANDS2 + BANDS3 
    self.FEATURES = BANDS1 + BANDS2 + BANDS3 + [self.RESPONSE]
    # Specify the size and shape of patches expected by the model.
    self.KERNEL_SIZE = 256
    self.KERNEL_SHAPE = [self.KERNEL_SIZE, self.KERNEL_SIZE]
    self.COLUMNS = [
      tf.io.FixedLenFeature(shape=self.KERNEL_SHAPE, dtype=tf.float32) for k in self.FEATURES
    ]
    self.FEATURES_DICT = dict(zip(self.FEATURES, self.COLUMNS))
    # Specify model training parameters.
    self.BATCH_SIZE = BATCH_SIZE
    self.EPOCHS = EPOCHS
    self.BUFFER_SIZE = 2000
    self.OPTIMIZER = 'adam'
    self.LOSS = LOSS
    self.dropout_prob = dropout_prob
    self.METRICS = ['AUC', "categorical_accuracy", metrics_.f1]
    self.image = image
    self.sam_arr = sam_arr



Writing tools/metrics_.py


In [ ]:
%%writefile {PACKAGE_PATH}/preprocessing.py

import tensorflow as tf
import ee

__all__ = ["Preprocessor", "maskL8sr", "EnsureTwodigit", "GenSeasonalDatesMonthly"]

class Preprocessor:
  """
  Class that preprocessese and returns the training, evaluation and testing data from google cloud bucket
  """
  def __init__(self, config):
    self.config = config

  def parse_tfrecord(self, example_proto):
    """
    The parsing function Read a serialized example into the structure defined by FEATURES_DICT.
  
    Parameters
    ----------
    example_proto: a serialized Example

    Returns
    ----------
    A dictionary of tensors, keyed by feature name.

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    return tf.io.parse_single_example(example_proto, self.config.FEATURES_DICT)


  def to_tuple(self, inputs):
    """
    Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
    Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
    Parameters
    ----------
    inputs: A dictionary of tensors, keyed by feature name.

    Returns
    ----------
    A tuple of (inputs, outputs).

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    inputsList = [inputs.get(key) for key in self.config.FEATURES]
    stacked = tf.stack(inputsList, axis=0)
    # Convert from CHW to HWC
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked[:,:,:len(self.config.BANDS)], tf.reshape(tf.one_hot(tf.cast(stacked[:,:,len(self.config.BANDS):], tf.int32), depth=2),[256,256,2])


  def get_dataset(self, pattern):
    """
    Function to read, parse and format to tuple a set of input tfrecord files.
    Get all the files matching the pattern, parse and convert to tuple.
    Parameters
    ----------
    pattern: A file pattern to match in a Cloud Storage bucket.

    Returns
    ----------
    A tf.data.Dataset

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    try:
      glob = tf.io.gfile.glob(pattern)
    except:
      # print("the bucket you specified doesn't exist")
      return "the bucket you specified doesn't exist"
    # glob = tf.io.gfile.glob(pattern)
    if glob == []:
      return "the path you specified doesn't have the data"
    dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
    dataset = dataset.map(self.parse_tfrecord, num_parallel_calls=5)
    dataset = dataset.map(self.to_tuple, num_parallel_calls=5)
    return dataset

  def get_training_dataset(self, location):
    """
    Get the preprocessed training dataset
    Parameters
    ----------
    location: string

    Returns
    ----------
    A tf.data.Dataset of training data.

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + "training_patches_" + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    dataset = dataset.shuffle(self.config.BUFFER_SIZE).batch(self.config.BATCH_SIZE).repeat()
    return dataset

  def get_training_dataset_for_testing(self, location):
    """
    Get the preprocessed training dataset for testing
    Parameters
    ----------
    location: string

    Returns
    ----------
    A tf.data.Dataset of training data.

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + "training_patches_" + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    if type(dataset) == str:
      return dataset
    dataset = dataset.batch(1).repeat()
    return dataset

  def get_eval_dataset(self, location):
    """
    Get the preprocessed evaluation dataset
    Parameters
    ----------
    location: string

    Returns
    ----------
    A tf.data.Dataset of evaluation data.

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + "eval_patches_" + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    if type(dataset) == str:
      return dataset
    dataset = dataset.batch(1).repeat()
    return dataset

  # print(iter(evaluation.take(1)).next())

  def get_test_dataset(self, location, test_base):
    """
    Get the preprocessed testing dataset
    Parameters
    ----------
    location: string

    Returns
    ----------
    A tf.data.Dataset of testing data.

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    glob = 'gs://' + self.config.BUCKET + '/' + location + test_base + '*'
    # print(glob)
    dataset = self.get_dataset(glob)
    if type(dataset) == str:
      return dataset
    dataset = dataset.batch(1).repeat()
    return dataset

def maskL8sr(image):
    """
    Get the landsat-8 image and returned a cloud masked image
    ----------
    image: ee.image.Image

    Returns
    ----------
    A maksed landsat-8 ee.image.Image

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
      qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask).select(BANDS).divide(10000)


def EnsureTwodigit(number):
  """
  Transform the input month into string in the
  correct format for date and time.
  ----------
  number: int

  Returns
  ----------
  months in string.

  """
  if number > 12:
    return str(12)
  if number < 10:
    return "0"+str(number)
  else:
    return str(number)

def GenSeasonalDatesMonthly(start, end, month_frequency = 3):
  """
  Given two dictionary containing the key month and year,
  return two arrays that contains the time between the 
  interval of start and end.
  ----------
  start: dict
  end: dict

  Returns
  ----------
  Two arrays containing the time elapsed between start and end

  """
  diff_year = end["year"] - start["year"]
  diff_month = end["month"] - start["month"]
  starts = []
  ends = []
  first_data = str(start["year"]) + "-" + EnsureTwodigit(start["month"]) + "-01"
  if diff_year > 0:
    return "please insert the same year"
  else:
    for i in range(round(diff_month/month_frequency)):
      first_data = str(start["year"]) + "-" + EnsureTwodigit(start["month"] + month_frequency * i) + "-01"
      second_data = str(start["year"]) + "-" + EnsureTwodigit(start["month"] + month_frequency * i + month_frequency) + "-01"
      starts.append(first_data)
      ends.append(second_data)
  return starts, ends


Writing tools/config.py


In [ ]:
%%writefile {PACKAGE_PATH}/losses_.py

import keras.backend as K
import tensorflow as tf
from keras.losses import categorical_crossentropy


__all__ = ["dice_coef", "dice_p_cc"]

def dice_coef(y_true, y_pred, smooth=1):
    """
    Recieve the true and predicted tensor and return the resulting dice loss
    to prevent overfitting.
    ----------
    y_true: tf.float32
    y_pred: tf.float32
    smooth: int/float

    Returns
    ----------
    A tf.float32 with same dimension as input tf.float32

    Notes
    -----
    The code is obtained/modified from:

    https://www.kaggle.com/code/kmader/u-net-with-dice-and-augmentation/notebook
    """
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)


def dice_p_cc(in_gt, in_pred):
    """
    Recieve the true and predicted tensor and return the resulting categorical dice loss
    ----------
    in_gt: tf.float32
    in_pred: tf.float32

    Returns
    ----------
    A tf.float32 with same dimension as input tf.float32

    Notes
    -----
    The code is obtained/modified from:

    https://www.kaggle.com/code/kmader/u-net-with-dice-and-augmentation/notebook
    """
    return categorical_crossentropy(in_gt, in_pred) - K.log(dice_coef(in_gt, in_pred))


Writing tools/preprocessing.py


In [ ]:
%%writefile {PACKAGE_PATH}/images.py

import json
from pprint import pprint
import numpy as np
import tqdm.notebook as tq
import tensorflow as tf
import ee
import subprocess
import ast


__all__ = ["doExport", "predictionSingleinput", "predictionMultipleinput", "predictionMultipleinput_3", \
           "uploadToGEEAsset", "LoadImage", "doPrediction_featurestack", \
            "doPrediction_multiview_2", "doPrediction_multiview_3"]


def doExport(out_image_base, kernel_buffer, region, setting, extra_folder= ""):
  """
  Export the image with features and area of interest
  to google cloud bucket. The function doesn't exit until
  the task is complete. The optional extra_folder arguement lets you put
  the exported tf.record.gz in a folder

  Parameters
  ----------
  out_image_base : string
  kernel_buffer : list
  region : ee.Geometry.BBox
  setting : dict
  extra_folder : string

  Notes
  -----
  The code is obtained/modified from:

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  if extra_folder == "":
    fileNamePrefix_ = setting.FOLDER + '/' + out_image_base
  else:
    fileNamePrefix_ = setting.FOLDER + '/' + extra_folder + '/' + out_image_base
  task = ee.batch.Export.image.toCloudStorage(
    image = setting.image.select(setting.BANDS),
    description = out_image_base,
    bucket = setting.BUCKET,
    fileNamePrefix = fileNamePrefix_,
    region = region.getInfo()['coordinates'],
    scale = 30,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': setting.KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')


def LoadImage(out_image_base, user_folder, kernel_buffer, setting, extra_folder = ""):
  """
  Load the image from the google cloud bucket and preprocess the image for prediction
  and provide crutial information for exporting to GEE asset

  Parameters
  ----------
  out_image_base : string
  user_folder : string
  kernel_buffer : list
  setting : dict
  extra_folder : string

  Notes
  -----
  The code is obtained/modified from:

  Returns
  ----------
  imageDataset as tensor data for prediction, patches as int, x_buffer as int, y_buffer as int, jsonFile as string

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  if extra_folder == "":
    process = subprocess.run(["gsutil", "ls", f'gs://{setting.BUCKET}/{setting.FOLDER}'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    filesList = process.stdout.split('\n')
  else:
    process = subprocess.run(["gsutil", "ls", f'gs://{setting.BUCKET}/{setting.FOLDER}/{extra_folder}'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    filesList = process.stdout.split('\n')

  print(filesList)
  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]
  print(exportFilesList)
  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  pprint(imageFilesList)
  print(jsonFile)
  if jsonFile == None:
    return "image path doesn't exist"

  # Load the contents of the mixer file to a JSON object.
  process = subprocess.run(["gsutil", "cat", f'{jsonFile}'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
  jsonText = process.stdout
  # Get a single string w/ newlines from the IPython.utils.text.SList
  # mixer = json.loads(jsonText.nlstr)
  mixer = ast.literal_eval(jsonText)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      setting.KERNEL_SHAPE[0] + kernel_buffer[0],
      setting.KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in setting.BANDS
  ]

  def parse_image(example_proto):
    """
    The parsing function Read a serialized example into the structure defined by FEATURES_DICT.

    Parameters
    ----------
    example_proto: a serialized Example

    Returns
    ----------
    A dictionary of tensors, keyed by feature name.

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    """
    Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
    Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
    Parameters
    ----------
    inputs: A dictionary of tensors, keyed by feature name.

    Returns
    ----------
    A tuple of (inputs, outputs).

    Notes
    -----
    The code is obtained/modified from:

    https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
    """
    inputsList = [inputs.get(key) for key in setting.BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

  imageFeaturesDict = dict(zip(setting.BANDS, imageColumns))
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)
  return imageDataset, patches, x_buffer, y_buffer, jsonFile

def predictionSingleinput(model, imageDataset, patches):
  """
  Given the model, and image for prediction, predict the image
  with feature stack U-Net

  Parameters
  ----------
  model : model : keras.engine.functional.Functional
  imageDataset : tf.data.Dataset of training data (BatchDataset)
  patches : int

  Returns
  ----------
  A tf.float32 with same dimension as imageDataset

  Notes
  -----
  The code is obtained/modified from:

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  print('Running predictions...')
  predictions = model.predict(imageDataset, steps=patches, verbose=1)
  return predictions

def predictionMultipleinput(model, imageDataset, patches, setting):
  """
  Given the model, and image for prediction, predict the image
  with Multi-view learning U-Net

  Parameters
  ----------
  model : model : keras.engine.functional.Functional
  imageDataset : tf.data.Dataset of training data (BatchDataset)
  patches : int
  setting : dict

  Returns
  ----------
  A tf.float32 with same dimension as imageDataset

  Notes
  -----
  The code is modified from:

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  print('Running predictions...')
  predictions = []
  pbar = tq.tqdm(total=patches)
  for data in imageDataset:
    pbar.update(1)
    x1, x2 = tf.split(data, [len(setting.BANDS1), len(setting.BANDS2)], 3)
    predictions.append(model.predict([x1, x2], verbose=0))
  return predictions

def predictionMultipleinput_3(model, imageDataset, patches, setting):
  """
  Given the model, and image for prediction, predict the image
  with Multi-view learning U-Net with 3 inputs

  Parameters
  ----------
  model : model : keras.engine.functional.Functional
  imageDataset : tf.data.Dataset of training data (BatchDataset)
  patches : int
  setting : dict

  Returns
  ----------
  A tf.float32 with same dimension as imageDataset

  Notes
  -----
  The code is obtained/modified from:

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  print('Running predictions...')
  predictions = []
  pbar = tq.tqdm(total=patches)
  for data in imageDataset:
    pbar.update(1)
    x1, x2, x3 = tf.split(data, [len(setting.BANDS1), len(setting.BANDS2), len(setting.BANDS3)], 3)
    predictions.append(model.predict([x1, x2, x3], verbose=0))
  return predictions

def uploadToGEEAsset(x_buffer, y_buffer, predictions, out_image_base, jsonFile, suffix, setting, multiview=False, user_folder='users/mewchayutaphong'):
  """
  Given the predictions, exported file location other information
  on the image to be exported, return the required information
  in order to export the predicted image to the GEE asset

  Parameters
  ----------
  x_buffer : int
  y_buffer : int
  predictions : tf.data.Dataset of training data (BatchDataset)
  out_image_base : string
  jsonFile : string
  suffix : string
  setting : dict
  user_folder : string

  Returns
  ----------
  A out_image_asset a location to the output GEE asset folder
  as a string, out_image_file of the prediction as TFRecord and the
  corresponding json file.

  Notes
  -----
  The code is obtained/modified from:

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  print('Writing predictions...')
  out_image_file = 'gs://' + setting.BUCKET + '/' + setting.FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    if multiview == True:
      predictionPatch = predictionPatch[0]
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+setting.KERNEL_SIZE, y_buffer:y_buffer+setting.KERNEL_SIZE]
    predictionPatch = np.argmax(predictionPatch, -1)
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()
 
  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base + suffix
  # !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}
  return out_image_asset, out_image_file, jsonFile


def doPrediction_featurestack(out_image_base, user_folder, kernel_buffer, model, suffix, setting, extra_folder=""):
  """
  Putting all the image functions together. Load the Image, predict the output with featurestack U-Net, return information
  ready to be exported to GEE asset
  ----------
  pattern: A file pattern to match in a Cloud Storage bucket.

  Returns
  ----------
  A out_image_asset a location to the output GEE asset folder
  as a string, out_image_file of the prediction as TFRecord and the
  corresponding json file.

  Notes
  -----
  The code is obtained/modified from:

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  output_load_image = LoadImage(out_image_base, user_folder, kernel_buffer, setting, extra_folder)
  if type(output_load_image) == str:
    return "wrong file location"
  imageDataset, patches, x_buffer, y_buffer, jsonFile = output_load_image
  predictions = predictionSingleinput(model, imageDataset, patches)
  out_image_asset, out_image_file, jsonFile = uploadToGEEAsset(x_buffer, y_buffer, predictions, out_image_base, jsonFile, suffix, setting, False)
  return out_image_asset, out_image_file, jsonFile

def doPrediction_multiview_2(out_image_base, user_folder, kernel_buffer, model, suffix, setting, extra_folder=""):
  """
  Putting all the image functions together. Load the Image, predict the output with 
  Multi-view U-Net (2 inputs), return information ready to be exported to GEE asset
  ----------
  pattern: A file pattern to match in a Cloud Storage bucket.

  Returns
  ----------
  A out_image_asset a location to the output GEE asset folder
  as a string, out_image_file of the prediction as TFRecord and the
  corresponding json file.

  Notes
  -----
  The code is obtained/modified from:

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  output_load_image = LoadImage(out_image_base, user_folder, kernel_buffer, setting, extra_folder)
  if type(output_load_image) == str:
    return "wrong file location"
  imageDataset, patches, x_buffer, y_buffer, jsonFile = output_load_image
  predictions = predictionMultipleinput(model, imageDataset, patches, setting)
  out_image_asset, out_image_file, jsonFile = uploadToGEEAsset(x_buffer, y_buffer, predictions, out_image_base, jsonFile, suffix, setting, True)
  return out_image_asset, out_image_file, jsonFile

def doPrediction_multiview_3(out_image_base, user_folder, kernel_buffer, model, suffix, setting, extra_folder=""):
  """
  Putting all the image functions together. Load the Image, predict the output with 
  Multi-view U-Net (3 inputs), return information ready to be exported to GEE asset
  ----------
  pattern: A file pattern to match in a Cloud Storage bucket.

  Returns
  ----------
  A out_image_asset a location to the output GEE asset folder
  as a string, out_image_file of the prediction as TFRecord and the
  corresponding json file.

  Notes
  -----
  The code is obtained/modified from:

  https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/UNET_regression_demo.ipynb
  """
  output_load_image = LoadImage(out_image_base, user_folder, kernel_buffer, setting, extra_folder)
  if type(output_load_image) == str:
    return "wrong file location"
  imageDataset, patches, x_buffer, y_buffer, jsonFile = output_load_image
  predictions = predictionMultipleinput_3(model, imageDataset, patches, setting)
  out_image_asset, out_image_file, jsonFile = uploadToGEEAsset(x_buffer, y_buffer, predictions, out_image_base, jsonFile, suffix, setting, True)
  return out_image_asset, out_image_file, jsonFile

Writing tools/losses_.py


## Authentication

Authentication with google colab, earth engine api and google cloud bucket is required before proceeding.

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

project_id = 'coastal-cell-299117'
!gcloud config set project {project_id}

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=T7CFfqdjSz3F0AtP5Yy2PFiuUyV7U2OY_vVU9MhGbCA&tc=Xfy8oshXohRLUFOc5DxOXg1XcmouU2F6kNLGmLmJ5AY&cc=dR0CLgL9jVm_b08qME-XzRv0gHLCfhUrQNa61xY2nng

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qgGlT3f1eiXCtfV7sa8U__0sY40ALqV3Jh6ScWd046qD2PDZHSP0-w

Successfully saved authorization token.
Updated property [core/project].


In [ ]:
from importlib import reload
import tensorflow as tf
import folium
from pprint import pprint
# reload(images) # Uncomment this line to rerun the modified packages
from tools import preprocessing, config, metrics_, losses_, images
from tensorflow.keras import losses

# Coding



## Objective 1: Predict water bodies at any location by selecting a model from GCB

- There are a total of 154 models stored in the GCB and the user can select their pretrained model from GCB. 
- User select a region of interest of where they want to predict the image which gets exported to GCB
- The image from GCB is imported into to do prediction
- Predicted image is exported to GEE asset

In [ ]:
start_date = '2018-01-01'
end_date = '2018-02-01'
# start_date = '2018-01-01'
# end_date = '2018-02-01'

# Sentinel-1 Data (10m)
S1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filterDate(start_date,end_date) \

S1A = S1.median()
S1 = S1.select('VV', 'VH').median()

# USG’s Landsat-8 Collection 1 and Tier 1 (30m)
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(start_date,end_date)

# Cloud masking function.
L8SR = l8sr.map(preprocessing.maskL8sr).median()

# NASADEM: NASA NASADEM Digital Elevation (30m)
elevation = ee.Image('NASA/NASADEM_HGT/001').select('elevation');
slope = ee.Terrain.slope(elevation);
aspect = ee.Terrain.aspect(elevation);

# JRC-Monthly Water history (30m)
waterdata = ee.ImageCollection('JRC/GSW1_3/MonthlyHistory').filterDate(start_date, end_date).median()
watermask = waterdata.select("water")
mask = watermask.gt(0) # masking out "no data" region
maskedComposite = waterdata.updateMask(mask).subtract(1) # Shifting the labels to make it binary

### Setting up Config - to train in Thailand

In total there are:
- 32 feature stack deep learning experiments
- 37 Multiview with 2 input experiments
- 8 Multiview with 3 input experiments

Each experiment has a different configuration for example, different experiment name, bands for each input layer. Hence, a configuration is neccesary. There is also configuration for training globally and is found in Preprocessing_and_export_global.ipynb notebook

In [ ]:
configs_fs = {}
train_size = 240*3
eval_size = 240*2

# Feature stack's 32 experiments

configs_fs["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle", "elevation", "slope", "aspect"], train_size, eval_size)
configs_fs["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle", "elevation"], train_size, eval_size)
configs_fs["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle", "slope"], train_size, eval_size)
configs_fs["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle", "aspect"], train_size, eval_size)
configs_fs["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle", "slope", "aspect"], train_size, eval_size)
configs_fs["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle", "elevation", "slope"], train_size, eval_size)
configs_fs["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle", "elevation", "aspect"], train_size, eval_size)

configs_fs["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH", "elevation", "slope", "aspect"], train_size, eval_size)
configs_fs["S1_el"] = config.configuration("S1_el", ["VV", "VH", "elevation"], train_size, eval_size)
configs_fs["S1_sl"] = config.configuration("S1_sl", ["VV", "VH", "slope"], train_size, eval_size)
configs_fs["S1_as"] = config.configuration("S1_as", ["VV", "VH", "aspect"], train_size, eval_size)
configs_fs["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH", "slope", "aspect"], train_size, eval_size)
configs_fs["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH", "elevation", "slope"], train_size, eval_size)
configs_fs["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH", "elevation", "aspect"], train_size, eval_size)

configs_fs["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope", "aspect"], train_size, eval_size)
configs_fs["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation"], train_size, eval_size)
configs_fs["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "slope"], train_size, eval_size)
configs_fs["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7", "aspect"], train_size, eval_size)
configs_fs["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "slope", "aspect"], train_size, eval_size)
configs_fs["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope"], train_size, eval_size)
configs_fs["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "aspect"], train_size, eval_size)

configs_fs["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "elevation"], train_size, eval_size)
configs_fs["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope"], train_size, eval_size)
configs_fs["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope","elevation", "aspect"], train_size, eval_size)

configs_fs["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "elevation"], train_size, eval_size)
configs_fs["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope"], train_size, eval_size)
configs_fs["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope","elevation", "aspect"], train_size, eval_size)

configs_fs["L8SR"] = config.configuration("L8SR", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size)
configs_fs["S1"] = config.configuration("S1", ["VV", "VH"], train_size, eval_size)
configs_fs["S1A"] = config.configuration("S1A", ["VV", "VH", "angle"], train_size, eval_size)
configs_fs["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH"], train_size, eval_size)
configs_fs["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle"], train_size, eval_size)

# Multi-view with 2 inputs learning's 37 experiments

configs_multi = {}

configs_multi["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2)
configs_multi["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle"], train_size, eval_size, ["elevation"], type_=2)
configs_multi["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle"], train_size, eval_size, ["slope"], type_=2)
configs_multi["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle"], train_size, eval_size, ["aspect"], type_=2)
configs_multi["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["slope", "aspect"], type_=2)
configs_multi["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope"], type_=2)
configs_multi["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "aspect"], type_=2)

configs_multi["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2)
configs_multi["S1_el"] = config.configuration("S1_el", ["VV", "VH"], train_size, eval_size, ["elevation"], type_=2)
configs_multi["S1_sl"] = config.configuration("S1_sl", ["VV", "VH"], train_size, eval_size, ["slope"], type_=2)
configs_multi["S1_as"] = config.configuration("S1_as", ["VV", "VH"], train_size, eval_size, ["aspect"], type_=2)
configs_multi["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH"], train_size, eval_size, ["slope", "aspect"], type_=2)
configs_multi["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH"], train_size, eval_size, ["elevation", "slope"], type_=2)
configs_multi["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH"], train_size, eval_size, ["elevation", "aspect"], type_=2)

configs_multi["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2)
configs_multi["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation"], type_=2)
configs_multi["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope"], type_=2)
configs_multi["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["aspect"], type_=2)
configs_multi["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope", "aspect"], type_=2)
configs_multi["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope"], type_=2)
configs_multi["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "aspect"], type_=2)


configs_multi["L8SR_S1_as"] = config.configuration("L8SR_S1_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "aspect"], type_=2)
configs_multi["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation"], type_=2)
configs_multi["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope"], type_=2)

configs_multi["L8SR_S1_sl_as"] = config.configuration("L8SR_S1_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope", "aspect"], type_=2)
configs_multi["L8SR_S1_el_sl"] = config.configuration("L8SR_S1_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "slope"], type_=2)
configs_multi["L8SR_S1_el_as"] = config.configuration("L8SR_S1_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "aspect"], type_=2)
configs_multi["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope","elevation", "aspect"], type_=2)

configs_multi["L8SR_S1A_as"] = config.configuration("L8SR_S1A_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "aspect"], type_=2)
configs_multi["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation"], type_=2)
configs_multi["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope"], type_=2)

configs_multi["L8SR_S1A_sl_as"] = config.configuration("L8SR_S1A_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope", "aspect"], type_=2)
configs_multi["L8SR_S1A_el_sl"] = config.configuration("L8SR_S1A_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "slope"], type_=2)
configs_multi["L8SR_S1A_el_as"] = config.configuration("L8SR_S1A_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "aspect"], type_=2)
configs_multi["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope","elevation", "aspect"], type_=2)

configs_multi["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], type_=2)
configs_multi["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], type_=2)

# Multi-view with 3 inputs learning's 8 experiments

configs_multi_3 = {}

configs_multi_3["L8SR_S1_as3"] = config.configuration("L8SR_S1_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["aspect"], type_=3)
configs_multi_3["L8SR_S1_el3"] = config.configuration("L8SR_S1_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["elevation"], type_=3)
configs_multi_3["L8SR_S1_sl3"] = config.configuration("L8SR_S1_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope"], type_=3)
configs_multi_3["L8SR_S1_sl_el_as3"] = config.configuration("L8SR_S1_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope","elevation", "aspect"], type_=3)

configs_multi_3["L8SR_S1A_as3"] = config.configuration("L8SR_S1A_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["aspect"], type_=3)
configs_multi_3["L8SR_S1A_el3"] = config.configuration("L8SR_S1A_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["elevation"], type_=3)
configs_multi_3["L8SR_S1A_sl3"] = config.configuration("L8SR_S1A_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope"], type_=3)
configs_multi_3["L8SR_S1A_sl_el_as3"] = config.configuration("L8SR_S1A_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope","elevation", "aspect"], type_=3)

### Setting up Config - to train globally

In total there are:
- 32 feature stack deep learning experiments
- 37 Multiview with 2 input experiments
- 8 Multiview with 3 input experiments

Each experiment has a different configuration for example, different experiment name, bands for each input layer. Hence, a configuration is neccesary. There is also configuration for training locally in Thailand and is found in Preprocessing_and_export.ipynb notebook

In [ ]:
configs_fs_global = {}
train_size = 72*10
eval_size = 72*3

#### Feature stack experiment

configs_fs_global["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_el"] = config.configuration("S1_el", ["VV", "VH", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["S1_sl"] = config.configuration("S1_sl", ["VV", "VH", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1_as"] = config.configuration("S1_as", ["VV", "VH", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7", "slope", "aspect"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "slope","elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "elevation"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle", "slope","elevation", "aspect"], train_size, eval_size, country = "global")

configs_fs_global["L8SR"] = config.configuration("L8SR", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, country = "global")
configs_fs_global["S1"] = config.configuration("S1", ["VV", "VH"], train_size, eval_size, country = "global")
configs_fs_global["S1A"] = config.configuration("S1A", ["VV", "VH", "angle"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH"], train_size, eval_size, country = "global")
configs_fs_global["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7", "VV", "VH", "angle"], train_size, eval_size, country = "global")

###### Multiexperiment

configs_multi_global = {}

configs_multi_global["S1A_el_sl_as"] = config.configuration("S1A_el_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1A_el"] = config.configuration("S1A_el", ["VV", "VH", "angle"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["S1A_sl"] = config.configuration("S1A_sl", ["VV", "VH", "angle"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["S1A_as"] = config.configuration("S1A_as", ["VV", "VH", "angle"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["S1A_sl_as"] = config.configuration("S1A_sl_as", ["VV", "VH", "angle"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1A_el_sl"] = config.configuration("S1A_el_sl", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["S1A_el_as"] = config.configuration("S1A_el_as", ["VV", "VH", "angle"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")

configs_multi_global["S1_el_sl_as"] = config.configuration("S1_el_sl_as", ["VV", "VH"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1_el"] = config.configuration("S1_el", ["VV", "VH"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["S1_sl"] = config.configuration("S1_sl", ["VV", "VH"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["S1_as"] = config.configuration("S1_as", ["VV", "VH"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["S1_sl_as"] = config.configuration("S1_sl_as", ["VV", "VH"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["S1_el_sl"] = config.configuration("S1_el_sl", ["VV", "VH"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["S1_el_as"] = config.configuration("S1_el_as", ["VV", "VH"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_el_sl_as"] = config.configuration("L8SR_el_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_el"] = config.configuration("L8SR_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation"], type_=2, country = "global")
configs_multi_global["L8SR_sl"] = config.configuration("L8SR_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope"], type_=2, country = "global")
configs_multi_global["L8SR_as"] = config.configuration("L8SR_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["aspect"], type_=2, country = "global")
configs_multi_global["L8SR_sl_as"] = config.configuration("L8SR_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_el_sl"] = config.configuration("L8SR_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_el_as"] = config.configuration("L8SR_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["elevation", "aspect"], type_=2, country = "global")


configs_multi_global["L8SR_S1_as"] = config.configuration("L8SR_S1_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el"] = config.configuration("L8SR_S1_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation"], type_=2, country = "global")
configs_multi_global["L8SR_S1_sl"] = config.configuration("L8SR_S1_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope"], type_=2, country = "global")

configs_multi_global["L8SR_S1_sl_as"] = config.configuration("L8SR_S1_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el_sl"] = config.configuration("L8SR_S1_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_S1_el_as"] = config.configuration("L8SR_S1_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "elevation", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1_sl_el_as"] = config.configuration("L8SR_S1_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "slope","elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_S1A_as"] = config.configuration("L8SR_S1A_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el"] = config.configuration("L8SR_S1A_el", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_sl"] = config.configuration("L8SR_S1A_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope"], type_=2, country = "global")

configs_multi_global["L8SR_S1A_sl_as"] = config.configuration("L8SR_S1A_sl_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el_sl"] = config.configuration("L8SR_S1A_el_sl", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "slope"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_el_as"] = config.configuration("L8SR_S1A_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "elevation", "aspect"], type_=2, country = "global")
configs_multi_global["L8SR_S1A_sl_el_as"] = config.configuration("L8SR_S1A_sl_el_as", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle", "slope","elevation", "aspect"], type_=2, country = "global")

configs_multi_global["L8SR_S1"] = config.configuration("L8SR_S1", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], type_=2, country = "global")
configs_multi_global["L8SR_S1A"] = config.configuration("L8SR_S1A", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], type_=2, country = "global")

configs_multi_3_global = {}

configs_multi_3_global["L8SR_S1_as3"] = config.configuration("L8SR_S1_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["aspect"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_el3"] = config.configuration("L8SR_S1_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["elevation"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_sl3"] = config.configuration("L8SR_S1_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1_sl_el_as3"] = config.configuration("L8SR_S1_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH"], ["slope","elevation", "aspect"], type_=3, country = "global")

configs_multi_3_global["L8SR_S1A_as3"] = config.configuration("L8SR_S1A_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["aspect"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_el3"] = config.configuration("L8SR_S1A_el3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["elevation"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_sl3"] = config.configuration("L8SR_S1A_sl3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope"], type_=3, country = "global")
configs_multi_3_global["L8SR_S1A_sl_el_as3"] = config.configuration("L8SR_S1A_sl_el_as3", ["B2", "B3", "B4", "B5", "B6", "B7"], train_size, eval_size, ["VV", "VH", "angle"], ["slope","elevation", "aspect"], type_=3)

In [ ]:
# Feature Stack

configs_fs["S1A_el_sl_as"].image = ee.Image.cat([S1A, elevation, slope, aspect]).float()
configs_fs["S1A_el"].image = ee.Image.cat([S1A, elevation]).float()
configs_fs["S1A_sl"].image = ee.Image.cat([S1A, slope]).float()
configs_fs["S1A_as"].image = ee.Image.cat([S1A, aspect]).float()
configs_fs["S1A_sl_as"].image = ee.Image.cat([S1A, slope, aspect]).float()
configs_fs["S1A_el_sl"].image = ee.Image.cat([S1A, elevation, slope]).float()
configs_fs["S1A_el_as"].image = ee.Image.cat([S1A, elevation, aspect]).float()

configs_fs["S1_el_sl_as"].image = ee.Image.cat([S1, elevation, slope, aspect]).float()
configs_fs["S1_el"].image = ee.Image.cat([S1, elevation]).float()
configs_fs["S1_sl"].image = ee.Image.cat([S1, slope]).float()
configs_fs["S1_as"].image = ee.Image.cat([S1, aspect]).float()
configs_fs["S1_sl_as"].image = ee.Image.cat([S1, slope, aspect]).float()
configs_fs["S1_el_sl"].image = ee.Image.cat([S1, elevation, slope]).float()
configs_fs["S1_el_as"].image = ee.Image.cat([S1, elevation, aspect]).float()

configs_fs["L8SR_el_sl_as"].image = ee.Image.cat([L8SR, elevation, slope, aspect]).float()
configs_fs["L8SR_el"].image = ee.Image.cat([L8SR, elevation]).float()
configs_fs["L8SR_sl"].image = ee.Image.cat([L8SR, slope]).float()
configs_fs["L8SR_as"].image = ee.Image.cat([L8SR, aspect]).float()
configs_fs["L8SR_sl_as"].image = ee.Image.cat([L8SR, slope, aspect]).float()
configs_fs["L8SR_el_sl"].image = ee.Image.cat([L8SR, elevation, slope]).float()
configs_fs["L8SR_el_as"].image = ee.Image.cat([L8SR, elevation, aspect]).float()

configs_fs["L8SR_S1_el"].image = ee.Image.cat([L8SR, S1, elevation]).float()
configs_fs["L8SR_S1_sl"].image = ee.Image.cat([L8SR, S1, slope]).float()
configs_fs["L8SR_S1_sl_el_as"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect]).float()

configs_fs["L8SR_S1A_el"].image = ee.Image.cat([L8SR, S1A, elevation]).float()
configs_fs["L8SR_S1A_sl"].image = ee.Image.cat([L8SR, S1A, slope]).float()
configs_fs["L8SR_S1A_sl_el_as"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect]).float()

configs_fs["L8SR"].image = L8SR.float()
configs_fs["S1"].image = S1.float()
configs_fs["S1A"].image = S1A.float()
configs_fs["L8SR_S1"].image = ee.Image.cat([L8SR, S1]).float()
configs_fs["L8SR_S1A"].image = ee.Image.cat([L8SR, S1A]).float()

# Multiview

configs_multi["S1A_el_sl_as"].image = ee.Image.cat([S1A, elevation, slope, aspect]).float() 
configs_multi["S1A_el"].image = ee.Image.cat([S1A, elevation]).float() 
configs_multi["S1A_sl"].image = ee.Image.cat([S1A, slope]).float() 
configs_multi["S1A_as"].image = ee.Image.cat([S1A, aspect]).float() 
configs_multi["S1A_sl_as"].image = ee.Image.cat([S1A, slope, aspect]).float() 
configs_multi["S1A_el_sl"].image = ee.Image.cat([S1A, elevation, slope]).float() 
configs_multi["S1A_el_as"].image = ee.Image.cat([S1A, elevation, aspect]).float() 

configs_multi["S1_el_sl_as"].image = ee.Image.cat([S1, elevation, slope, aspect]).float()  
configs_multi["S1_el"].image = ee.Image.cat([S1, elevation]).float()
configs_multi["S1_sl"].image = ee.Image.cat([S1, slope]).float() 
configs_multi["S1_as"].image = ee.Image.cat([S1, aspect]).float() 
configs_multi["S1_sl_as"].image = ee.Image.cat([S1, slope, aspect]).float()
configs_multi["S1_el_sl"].image = ee.Image.cat([S1, elevation, slope]).float() 
configs_multi["S1_el_as"].image = ee.Image.cat([S1, elevation, aspect]).float() 

configs_multi["L8SR_el_sl_as"].image = ee.Image.cat([L8SR, elevation, slope, aspect]).float() 
configs_multi["L8SR_el"].image = ee.Image.cat([L8SR, elevation]).float() 
configs_multi["L8SR_sl"].image = ee.Image.cat([L8SR, slope]).float() 
configs_multi["L8SR_as"].image = ee.Image.cat([L8SR, aspect]).float() 
configs_multi["L8SR_sl_as"].image = ee.Image.cat([L8SR, slope, aspect]).float() 
configs_multi["L8SR_el_sl"].image = ee.Image.cat([ L8SR, elevation, slope]).float() 
configs_multi["L8SR_el_as"].image = ee.Image.cat([L8SR, elevation, aspect]).float() 


configs_multi["L8SR_S1_as"].image = ee.Image.cat([L8SR, S1, aspect]).float()
configs_multi["L8SR_S1_el"].image = ee.Image.cat([L8SR, S1, elevation]).float() 
configs_multi["L8SR_S1_sl"].image = ee.Image.cat([L8SR, S1, slope]).float() 

configs_multi["L8SR_S1_sl_as"].image = ee.Image.cat([L8SR, S1, slope, aspect]).float() 
configs_multi["L8SR_S1_el_sl"].image = ee.Image.cat([L8SR, S1, elevation, slope]).float() 
configs_multi["L8SR_S1_el_as"].image = ee.Image.cat([L8SR, slope, elevation, aspect]).float() 
configs_multi["L8SR_S1_sl_el_as"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect]).float() 

configs_multi["L8SR_S1A_as"].image = ee.Image.cat([L8SR, S1A, aspect]).float()
configs_multi["L8SR_S1A_el"].image = ee.Image.cat([L8SR, S1A, elevation]).float() 
configs_multi["L8SR_S1A_sl"].image = ee.Image.cat([L8SR, S1A, slope]).float() 

configs_multi["L8SR_S1A_sl_as"].image = ee.Image.cat([L8SR, S1A, slope, aspect]).float() 
configs_multi["L8SR_S1A_el_sl"].image = ee.Image.cat([L8SR, S1A, elevation, slope]).float() 
configs_multi["L8SR_S1A_el_as"].image = ee.Image.cat([L8SR, S1A, elevation, aspect]).float() 
configs_multi["L8SR_S1A_sl_el_as"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect]).float() 

configs_multi["L8SR_S1"].image = ee.Image.cat([L8SR, S1]).float() 
configs_multi["L8SR_S1A"].image = ee.Image.cat([L8SR, S1A]).float() 

# Multiview-3


configs_multi_3["L8SR_S1_as3"].image = ee.Image.cat([L8SR, S1, aspect]).float()
configs_multi_3["L8SR_S1_el3"].image = ee.Image.cat([L8SR, S1, elevation]).float()
configs_multi_3["L8SR_S1_sl3"].image = ee.Image.cat([L8SR, S1, slope]).float()
configs_multi_3["L8SR_S1_sl_el_as3"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect]).float()

configs_multi_3["L8SR_S1A_as3"].image = ee.Image.cat([L8SR, S1A, aspect]).float()
configs_multi_3["L8SR_S1A_el3"].image = ee.Image.cat([L8SR, S1A, elevation]).float()
configs_multi_3["L8SR_S1A_sl3"].image = ee.Image.cat([L8SR, S1A, slope]).float()
configs_multi_3["L8SR_S1A_sl_el_as3"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect]).float()

In [ ]:
# Feature Stack

configs_fs_global["S1A_el_sl_as"].image = ee.Image.cat([S1A, elevation, slope, aspect]).float()
configs_fs_global["S1A_el"].image = ee.Image.cat([S1A, elevation]).float()
configs_fs_global["S1A_sl"].image = ee.Image.cat([S1A, slope]).float()
configs_fs_global["S1A_as"].image = ee.Image.cat([S1A, aspect]).float()
configs_fs_global["S1A_sl_as"].image = ee.Image.cat([S1A, slope, aspect]).float()
configs_fs_global["S1A_el_sl"].image = ee.Image.cat([S1A, elevation, slope]).float()
configs_fs_global["S1A_el_as"].image = ee.Image.cat([S1A, elevation, aspect]).float()

configs_fs_global["S1_el_sl_as"].image = ee.Image.cat([S1, elevation, slope, aspect]).float()
configs_fs_global["S1_el"].image = ee.Image.cat([S1, elevation]).float()
configs_fs_global["S1_sl"].image = ee.Image.cat([S1, slope]).float()
configs_fs_global["S1_as"].image = ee.Image.cat([S1, aspect]).float()
configs_fs_global["S1_sl_as"].image = ee.Image.cat([S1, slope, aspect]).float()
configs_fs_global["S1_el_sl"].image = ee.Image.cat([S1, elevation, slope]).float()
configs_fs_global["S1_el_as"].image = ee.Image.cat([S1, elevation, aspect]).float()

configs_fs_global["L8SR_el_sl_as"].image = ee.Image.cat([L8SR, elevation, slope, aspect]).float()
configs_fs_global["L8SR_el"].image = ee.Image.cat([L8SR, elevation]).float()
configs_fs_global["L8SR_sl"].image = ee.Image.cat([L8SR, slope]).float()
configs_fs_global["L8SR_as"].image = ee.Image.cat([L8SR, aspect]).float()
configs_fs_global["L8SR_sl_as"].image = ee.Image.cat([L8SR, slope, aspect]).float()
configs_fs_global["L8SR_el_sl"].image = ee.Image.cat([L8SR, elevation, slope]).float()
configs_fs_global["L8SR_el_as"].image = ee.Image.cat([L8SR, elevation, aspect]).float()

configs_fs_global["L8SR_S1_el"].image = ee.Image.cat([L8SR, S1, elevation]).float()
configs_fs_global["L8SR_S1_sl"].image = ee.Image.cat([L8SR, S1, slope]).float()
configs_fs_global["L8SR_S1_sl_el_as"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect]).float()

configs_fs_global["L8SR_S1A_el"].image = ee.Image.cat([L8SR, S1A, elevation]).float()
configs_fs_global["L8SR_S1A_sl"].image = ee.Image.cat([L8SR, S1A, slope]).float()
configs_fs_global["L8SR_S1A_sl_el_as"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect]).float()

configs_fs_global["L8SR"].image = L8SR.float()
configs_fs_global["S1"].image = S1.float()
configs_fs_global["S1A"].image = S1A.float()
configs_fs_global["L8SR_S1"].image = ee.Image.cat([L8SR, S1]).float()
configs_fs_global["L8SR_S1A"].image = ee.Image.cat([L8SR, S1A]).float()


# Multiview

configs_multi_global["S1A_el_sl_as"].image = ee.Image.cat([S1A, elevation, slope, aspect]).float() 
configs_multi_global["S1A_el"].image = ee.Image.cat([S1A, elevation]).float() 
configs_multi_global["S1A_sl"].image = ee.Image.cat([S1A, slope]).float() 
configs_multi_global["S1A_as"].image = ee.Image.cat([S1A, aspect]).float() 
configs_multi_global["S1A_sl_as"].image = ee.Image.cat([S1A, slope, aspect]).float() 
configs_multi_global["S1A_el_sl"].image = ee.Image.cat([S1A, elevation, slope]).float() 
configs_multi_global["S1A_el_as"].image = ee.Image.cat([S1A, elevation, aspect]).float() 

configs_multi_global["S1_el_sl_as"].image = ee.Image.cat([S1, elevation, slope, aspect]).float()  
configs_multi_global["S1_el"].image = ee.Image.cat([S1, elevation]).float()
configs_multi_global["S1_sl"].image = ee.Image.cat([S1, slope]).float() 
configs_multi_global["S1_as"].image = ee.Image.cat([S1, aspect]).float() 
configs_multi_global["S1_sl_as"].image = ee.Image.cat([S1, slope, aspect]).float()
configs_multi_global["S1_el_sl"].image = ee.Image.cat([S1, elevation, slope]).float() 
configs_multi_global["S1_el_as"].image = ee.Image.cat([S1, elevation, aspect]).float() 

configs_multi_global["L8SR_el_sl_as"].image = ee.Image.cat([L8SR, elevation, slope, aspect]).float() 
configs_multi_global["L8SR_el"].image = ee.Image.cat([L8SR, elevation]).float() 
configs_multi_global["L8SR_sl"].image = ee.Image.cat([L8SR, slope]).float() 
configs_multi_global["L8SR_as"].image = ee.Image.cat([L8SR, aspect]).float() 
configs_multi_global["L8SR_sl_as"].image = ee.Image.cat([L8SR, slope, aspect]).float() 
configs_multi_global["L8SR_el_sl"].image = ee.Image.cat([ L8SR, elevation, slope]).float() 
configs_multi_global["L8SR_el_as"].image = ee.Image.cat([L8SR, elevation, aspect]).float() 


configs_multi_global["L8SR_S1_as"].image = ee.Image.cat([L8SR, S1, aspect]).float()
configs_multi_global["L8SR_S1_el"].image = ee.Image.cat([L8SR, S1, elevation]).float() 
configs_multi_global["L8SR_S1_sl"].image = ee.Image.cat([L8SR, S1, slope]).float() 

configs_multi_global["L8SR_S1_sl_as"].image = ee.Image.cat([L8SR, S1, slope, aspect]).float() 
configs_multi_global["L8SR_S1_el_sl"].image = ee.Image.cat([L8SR, S1, elevation, slope]).float() 
configs_multi_global["L8SR_S1_el_as"].image = ee.Image.cat([L8SR, slope, elevation, aspect]).float() 
configs_multi_global["L8SR_S1_sl_el_as"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect]).float() 

configs_multi_global["L8SR_S1A_as"].image = ee.Image.cat([L8SR, S1A, aspect]).float()
configs_multi_global["L8SR_S1A_el"].image = ee.Image.cat([L8SR, S1A, elevation]).float() 
configs_multi_global["L8SR_S1A_sl"].image = ee.Image.cat([L8SR, S1A, slope]).float() 

configs_multi_global["L8SR_S1A_sl_as"].image = ee.Image.cat([L8SR, S1A, slope, aspect]).float() 
configs_multi_global["L8SR_S1A_el_sl"].image = ee.Image.cat([L8SR, S1A, elevation, slope]).float() 
configs_multi_global["L8SR_S1A_el_as"].image = ee.Image.cat([L8SR, S1A, elevation, aspect]).float() 
configs_multi_global["L8SR_S1A_sl_el_as"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect]).float() 

configs_multi_global["L8SR_S1"].image = ee.Image.cat([L8SR, S1]).float() 
configs_multi_global["L8SR_S1A"].image = ee.Image.cat([L8SR, S1A]).float() 

# Multiview-3


configs_multi_3_global["L8SR_S1_as3"].image = ee.Image.cat([L8SR, S1, aspect])
configs_multi_3_global["L8SR_S1_el3"].image = ee.Image.cat([L8SR, S1, elevation])
configs_multi_3_global["L8SR_S1_sl3"].image = ee.Image.cat([L8SR, S1, slope])
configs_multi_3_global["L8SR_S1_sl_el_as3"].image = ee.Image.cat([L8SR, S1, slope, elevation, aspect])

configs_multi_3_global["L8SR_S1A_as3"].image = ee.Image.cat([L8SR, S1A, aspect])
configs_multi_3_global["L8SR_S1A_el3"].image = ee.Image.cat([L8SR, S1A, elevation])
configs_multi_3_global["L8SR_S1A_sl3"].image = ee.Image.cat([L8SR, S1A, slope])
configs_multi_3_global["L8SR_S1A_sl_el_as3"].image = ee.Image.cat([L8SR, S1A, slope, elevation, aspect])


### Export Images




Initially we load the model of interest, and configure where we export the image of interest. For demonstration purposes we selected 10 regions in
- Thailand
- Tibet
- Ghana
- brazil
- mexico
- pakistan
- egypt
- cambodia
- India

In [ ]:
# Output assets folder: YOUR FOLDER
user_folder = 'users/mewchayutaphong' # INSERT YOUR FOLDER HERE.

# Half this will extend on the sides of each patch.
kernel_buffer = [128, 128]

th_image_base = f'Thailand_with_Multi_'
th_region = ee.Geometry.BBox(100.30632852425321, 17.709225431372587, 100.74128946175321, 18.20417872756825)

tb_image_base = f'Tibet_with_Multi_'
tb_region = ee.Geometry.BBox(83.7866460908476, 31.02991423438545, 84.4782964814726, 31.623526673040716)

gm_image_base = f'Ghana_with_Multi_'
gm_region = ee.Geometry.BBox(-1.9983132238272127, 5.925449378444892, -1.8517086095694002, 6.081004042776062)

brazil_image_base = f'brazil_with_Multi_'
brazil_region = ee.Geometry.BBox(-56.914136208542024, -27.726731964448247, -55.734106911667024, -27.158518840235534)

mexico_image_base = f'mexico_with_Multi_'
mexico_region = ee.Geometry.BBox(-93.99463576763688, 16.864181463158733, -93.24756545513688, 17.352438403346977)

pakistan_image_base = f'pakistan_with_Multi_'
pakistan_region = ee.Geometry.BBox(70.62737898109093, 29.031658472574723, 70.86907819984093, 29.247564938104578)

egypt_image_base = f'egypt_with_Multi_'
egypt_region = ee.Geometry.BBox(32.25680416139795, 22.776830239482934, 33.28951900514795, 23.267354217706654)

cambodia_image_base = f'cambodia_with_Multi_'
cambodia_region = ee.Geometry.BBox(103.7086527335668, 12.395622357749016, 104.5710794913793, 13.2151653909455)

India_image_base = f'India_with_Multi_'
India_region = ee.Geometry.BBox(80.89907945596022, 23.91896359428029, 81.32754625283522, 24.24995005933742)

Bangladesh_image_base = f'Bangladesh_with_Multi_'
Bangladesh_region = ee.Geometry.BBox(89.25478625914253, 24.205326578074743, 90.15566516539253, 24.765228419516816)

#### From model trained in thailand

In [ ]:
## Here we select the model of interest
conf = configs_multi["L8SR_S1"]
preproc = preprocessing.Preprocessor(conf)
MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)

## Model is loaded
model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "custom_accuracy": metrics_.custom_accuracy})

## Export the image to GCB before prediction
images.doExport(th_image_base, kernel_buffer, th_region, conf)

## load the image from GCB and predict the water bodies
## Below gives a choice of the type of model interested 
## Please use doPrediction_featurestack for all FS U-Net model, doPrediction_multiview_2 for M2 U-Net
## doPrediction_multiview_3 for M3 U-Net

# out_image_asset, out_image_file, jsonFile = doPrediction_featurestack(th_image_base, user_folder, kernel_buffer, model_custom, "_epochs_10_", conf)
out_image_asset, out_image_file, jsonFile = images.doPrediction_multiview_2(th_image_base, user_folder, kernel_buffer, model_custom, "multiview-2_epochs_10_", conf)
# out_image_asset, out_image_file, jsonFile = doPrediction_multiview_3(th_image_base, user_folder, kernel_buffer, model_custom, "_epochs_10_", conf)

## Upload image to GEE
!earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}


Running image export to Cloud Storage...
Image export completed.
Looking for TFRecord files...
['gs://geebucketwater/m2_TH_Cnn_L8SR_S1/', 'gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Ghana_with_MultiL8SR_S1.TFRecord', 'gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Ghana_with_MultiL8SR_S1.json', 'gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Ghana_with_MultiL8SR_S1.tfrecord.gz', 'gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Thailand_with_Multi_00000.tfrecord.gz', 'gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Thailand_with_Multi_00001.tfrecord.gz', 'gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Thailand_with_Multi_mixer.json', 'gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Models/', '']
['gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Thailand_with_Multi_00000.tfrecord.gz', 'gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Thailand_with_Multi_00001.tfrecord.gz', 'gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Thailand_with_Multi_mixer.json']
['gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Thailand_with_Multi_00000.tfrecord.gz',
 'gs://geebucketwater/m2_TH_Cnn_L8SR_S1/Thailan

  0%|          | 0/42 [00:00<?, ?it/s]

Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Writing patch 4...
Writing patch 5...
Writing patch 6...
Writing patch 7...
Writing patch 8...
Writing patch 9...
Writing patch 10...
Writing patch 11...
Writing patch 12...
Writing patch 13...
Writing patch 14...
Writing patch 15...
Writing patch 16...
Writing patch 17...
Writing patch 18...
Writing patch 19...
Writing patch 20...
Writing patch 21...
Writing patch 22...
Writing patch 23...
Writing patch 24...
Writing patch 25...
Writing patch 26...
Writing patch 27...
Writing patch 28...
Writing patch 29...
Writing patch 30...
Writing patch 31...
Writing patch 32...
Writing patch 33...
Writing patch 34...
Writing patch 35...
Writing patch 36...
Writing patch 37...
Writing patch 38...
Writing patch 39...
Writing patch 40...
Writing patch 41...
Started upload task with ID: J6HDIKD6WIIASO6EWFJPTH4Y


#### From model trained globally

In [ ]:
## Here we select the model of interest from the global configuration
conf = configs_multi_global["L8SR_S1"]
preproc = preprocessing.Preprocessor(conf)
MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)

## Model is loaded
model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "custom_accuracy": metrics_.custom_accuracy})

## Export the image to GCB before prediction
images.doExport(th_image_base, kernel_buffer, th_region, conf)

## load the image from GCB and predict the water bodies
## Below gives a choice of the type of model interested 
## Please use doPrediction_featurestack for all FS U-Net model, doPrediction_multiview_2 for M2 U-Net
## doPrediction_multiview_3 for M3 U-Net

# out_image_asset, out_image_file, jsonFile = doPrediction_featurestack(th_image_base, user_folder, kernel_buffer, model_custom, "_epochs_10_", conf)
out_image_asset, out_image_file, jsonFile = images.doPrediction_multiview_2(th_image_base, user_folder, kernel_buffer, model_custom, "multiview-2_epochs_10_", conf)
# out_image_asset, out_image_file, jsonFile = doPrediction_multiview_3(th_image_base, user_folder, kernel_buffer, model_custom, "_epochs_10_", conf)

## Upload image to GEE
!earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}


## Objective 2: Dem Experiment

In this experiment, the S1, L8SR, S1_sl, L8SR + (S1_sl) and L8SR + S1A + sl model will be compared against each other through predicting the image at Tibet during Jan-Feb 2018.

This will enable us to visualize the effect of adding feature from the digital elevation model on landsat-8 and sentinel-1. In particular, slope will improve the prediction of S1 and if L8SR is paired through multiple views the performance would also improve

In [ ]:
# DEM Experiment in Tibet
# S1 (0.61)
# L8SR (0.94)
# S1 - sl (0.96) 
# L8SR+(S1-sl) (0.97) 
# L8SR+S1A+sl (0.96)

tb_region = ee.Geometry.BBox(83.7866460908476, 31.02991423438545, 84.4782964814726, 31.623526673040716)

DEM_EXP = [configs_fs_global["S1"], configs_fs_global["L8SR"], \
           configs_fs_global["S1_sl"], configs_multi_global["L8SR_S1_sl"], configs_multi_3_global["L8SR_S1A_sl3"]]

TRAIN_SIZE = 72*10
EVAL_SIZE = 72*3

for i, conf in enumerate(DEM_EXP):
  print(i)
  preproc = preprocessing.Preprocessor(conf)
  MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE + "_EPOCHS_" + str(10)
  model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1})
  tb_image_base = f'Tibet_with_Multi_{conf.PROJECT_TITLE}'
  images.doExport(tb_image_base, kernel_buffer, tb_region, conf)
  if conf.type_ == "fs":
    out_image_asset, out_image_file, jsonFile = images.doPrediction_featurestack(tb_image_base, user_folder, \
                              kernel_buffer, model_custom, "DEM_EXP", conf)
  if conf.type_ == "m2":
    out_image_asset, out_image_file, jsonFile = images.doPrediction_multiview_2(tb_image_base, user_folder, \
                            kernel_buffer, model_custom, "DEM_EXP", conf)
  if conf.type_ == "m3":
    out_image_asset, out_image_file, jsonFile = images.doPrediction_multiview_3(tb_image_base, user_folder, \
                            kernel_buffer, model_custom, "DEM_EXP", conf)
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}
  
  

0


Running image export to Cloud Storage...
Image export completed.
Looking for TFRecord files...
['gs://geebucketwater/fs_global_Cnn_S1/', 'gs://geebucketwater/fs_global_Cnn_S1/Tibet_with_Multi_.TFRecord', 'gs://geebucketwater/fs_global_Cnn_S1/Tibet_with_Multi_.json', 'gs://geebucketwater/fs_global_Cnn_S1/Tibet_with_Multi_.tfrecord.gz', 'gs://geebucketwater/fs_global_Cnn_S1/Tibet_with_Multi_S1.TFRecord', 'gs://geebucketwater/fs_global_Cnn_S1/Tibet_with_Multi_S1.json', 'gs://geebucketwater/fs_global_Cnn_S1/Tibet_with_Multi_S1.tfrecord.gz', 'gs://geebucketwater/fs_global_Cnn_S1/Models/', '']
['gs://geebucketwater/fs_global_Cnn_S1/Tibet_with_Multi_S1.TFRecord', 'gs://geebucketwater/fs_global_Cnn_S1/Tibet_with_Multi_S1.json', 'gs://geebucketwater/fs_global_Cnn_S1/Tibet_with_Multi_S1.tfrecord.gz']
['gs://geebucketwater/fs_global_Cnn_S1/Tibet_with_Multi_S1.tfrecord.gz']
gs://geebucketwater/fs_global_Cnn_S1/Tibet_with_Multi_S1.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 10,
 'project

KeyboardInterrupt: ignored

## Objective 3: Seasonal Effect experiment - GlobalWaterNet

In this section, the GlobalWaterNet is used to predict in the same region as in objective 2, but in this case the model will predict seasonally from jan-april, april-july, july-oct, oct-dec. 

This will enable user to gain insight into the water bodies through time. In addition, it will show that the points which has no-data isn't predicted incorrectly (this can be seen in the paper).

In [ ]:
# Output assets folder: YOUR FOLDER
user_folder = 'users/mewchayutaphong' # INSERT YOUR FOLDER HERE.

# Half this will extend on the sides of each patch.
kernel_buffer = [128, 128]

tb_region = ee.Geometry.BBox(83.7866460908476, 31.02991423438545, 84.4782964814726, 31.623526673040716)



### Loading the model
TRAIN_SIZE = 72*10
EVAL_SIZE = 72*3
configs_multi_global["L8SR_S1A_sl_CCDICE_dp0.3"] = config.configuration("L8SR_S1A_sl_CCDICE_dp0.3", BANDS1 = ["B2", "B3", "B4", "B5", "B6", "B7"], BANDS2=["VV", "VH", "angle", "slope"]\
                                                 , TRAIN_SIZE= TRAIN_SIZE, EVAL_SIZE = EVAL_SIZE\
                                                 , EPOCHS=10, BATCH_SIZE = 16, dropout_prob=0.3, LOSS=losses_.dice_p_cc, type_ = 2, country="Global")
conf = configs_multi_global["L8SR_S1A_sl_CCDICE_dp0.3"]
preproc = preprocessing.Preprocessor(conf)
MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE
model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "dice_p_cc": losses_.dice_p_cc})

### Observing seasonal effect
start_dates, end_dates = preprocessing.GenSeasonalDatesMonthly({"year": 2018, "month": 1}, {"year": 2018, "month": 12})
tb_image_base = f'Tibet_with_Multi_seasonalEXP_'
for i in range(len(start_dates)):
  S1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
          .filterDate(start_dates[i], end_dates[i]) \

  S1A = S1.median()
  l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(start_dates[i], end_dates[i])
  L8SR = l8sr.map(preprocessing.maskL8sr).median()
  configs_multi_global["L8SR_S1A_sl_CCDICE_dp0.3"].image = ee.Image.cat([L8SR, S1A, slope]).float()
  images.doExport(tb_image_base, kernel_buffer, tb_region, conf, "season_"+str(i))
  out_image_asset, out_image_file, jsonFile = images.doPrediction_multiview_2(tb_image_base, user_folder, \
                          kernel_buffer, model_custom, str(i), conf, "season_"+str(i))
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}


Running image export to Cloud Storage...
Image export completed.
Looking for TFRecord files...
['gs://geebucketwater/m2_Global_Cnn_L8SR_S1A_sl_CCDICE_dp0.3/season_0/Tibet_with_Multi_seasonalEXP_00000.tfrecord.gz',
 'gs://geebucketwater/m2_Global_Cnn_L8SR_S1A_sl_CCDICE_dp0.3/season_0/Tibet_with_Multi_seasonalEXP_00001.tfrecord.gz',
 'gs://geebucketwater/m2_Global_Cnn_L8SR_S1A_sl_CCDICE_dp0.3/season_0/Tibet_with_Multi_seasonalEXP_00002.tfrecord.gz',
 'gs://geebucketwater/m2_Global_Cnn_L8SR_S1A_sl_CCDICE_dp0.3/season_0/Tibet_with_Multi_seasonalEXP_00003.tfrecord.gz',
 'gs://geebucketwater/m2_Global_Cnn_L8SR_S1A_sl_CCDICE_dp0.3/season_0/Tibet_with_Multi_seasonalEXP_00004.tfrecord.gz']
gs://geebucketwater/m2_Global_Cnn_L8SR_S1A_sl_CCDICE_dp0.3/season_0/Tibet_with_Multi_seasonalEXP_mixer.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 10,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                               

  0%|          | 0/80 [00:00<?, ?it/s]

Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Writing patch 4...
Writing patch 5...
Writing patch 6...
Writing patch 7...
Writing patch 8...
Writing patch 9...
Writing patch 10...
Writing patch 11...
Writing patch 12...
Writing patch 13...
Writing patch 14...
Writing patch 15...
Writing patch 16...
Writing patch 17...
Writing patch 18...
Writing patch 19...
Writing patch 20...
Writing patch 21...
Writing patch 22...
Writing patch 23...
Writing patch 24...
Writing patch 25...
Writing patch 26...
Writing patch 27...
Writing patch 28...
Writing patch 29...
Writing patch 30...
Writing patch 31...
Writing patch 32...
Writing patch 33...
Writing patch 34...
Writing patch 35...
Writing patch 36...
Writing patch 37...
Writing patch 38...
Writing patch 39...
Writing patch 40...
Writing patch 41...
Writing patch 42...
Writing patch 43...
Writing patch 44...
Writing patch 45...
Writing patch 46...
Writing patch 47...
Writing patch 48...
Writing

## Objective 4: Flooding Experiment - THWaterNet

Flooding is common in Thailand due to seasonal monsoon rainfall. The THWaterNet is used to study the flooding extent during dec 2016 to early 2017.

Consequently, the result shows that the flood not only affect the built-ups but also cropland and hinders human activies. (More in the paper)

In [ ]:
from tensorflow.keras import losses
# Output assets folder: YOUR FOLDER
user_folder = 'users/mewchayutaphong' # INSERT YOUR FOLDER HERE.

# Half this will extend on the sides of each patch.
kernel_buffer = [128, 128]

# tb_region = ee.Geometry.BBox(83.7866460908476, 31.02991423438545, 84.4782964814726, 31.623526673040716)
# flood_region = ee.Geometry.BBox(100.0468738016213, 7.634771858293297, 100.1423175272073, 7.749770688005384)
flood_region = ee.Geometry.BBox(99.89924501744167, 7.634771858293311, 100.22265505162136, 7.876981296784481)


### Loading the model
TRAIN_SIZE = 72*10
EVAL_SIZE = 72*3
configs_multi_global["L8SR_S1A_sl_CC_dp0.3"] = config.configuration("L8SR_S1A_sl_CC_dp0.3", BANDS1 = ["B2", "B3", "B4", "B5", "B6", "B7"], BANDS2=["VV", "VH", "angle", "slope"]\
                                                 , TRAIN_SIZE= TRAIN_SIZE, EVAL_SIZE = EVAL_SIZE\
                                                 , EPOCHS=10, BATCH_SIZE = 16, dropout_prob=0.3, LOSS=losses.get("categorical_crossentropy"), type_ = 2, country="TH")
conf = configs_multi_global["L8SR_S1A_sl_CC_dp0.3"]
preproc = preprocessing.Preprocessor(conf)
MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE
model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "dice_p_cc": losses_.dice_p_cc})

### Observing seasonal effect
# start_dates = ["2016-11-01", "2016-12-01", "2017-01-01", "2017-02-01", "2017-03-01"]
# end_dates = ["2016-12-01", "2017-01-01", "2017-02-01", "2017-03-01", "2017-04-01"]
start_dates = ["2016-12-01", "2016-12-15", "2017-01-01", "2017-01-15"]
end_dates = ["2016-12-15", "2016-12-30", "2017-01-15", "2017-01-30"]
# start_dates, end_dates = GenSeasonalDatesMonthly({"year": 2016, "month": 1}, {"year": 2018, "month": 12})
flood_image_base = f'flood_thai_with_Multi_floodEXP_half_'
for i in range(len(start_dates)):
  if i == 2:
    break
  print(start_dates[i], end_dates[i])
  S1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
          .filterDate(start_dates[i], end_dates[i]) \

  S1A = S1.median()
  l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(start_dates[i], end_dates[i])
  L8SR = l8sr.map(preprocessing.maskL8sr).median()
  configs_multi_global["L8SR_S1A_sl_CC_dp0.3"].image = ee.Image.cat([L8SR, S1A, slope]).float()
  images.doExport(flood_image_base, kernel_buffer, flood_region, conf, "flood_half"+str(i))
  out_image_asset, out_image_file, jsonFile = images.doPrediction_multiview_2(flood_image_base, user_folder, \
                          kernel_buffer, model_custom, str(i), conf, "flood_half"+str(i))
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}


2016-12-01 2016-12-15
Running image export to Cloud Storage...
Image export completed.
Looking for TFRecord files...
['gs://geebucketwater/m2_TH_Cnn_L8SR_S1A_sl_CC_dp0.3/flood_half0/flood_thai_with_Multi_floodEXP_half_.tfrecord.gz']
gs://geebucketwater/m2_TH_Cnn_L8SR_S1A_sl_CC_dp0.3/flood_half0/flood_thai_with_Multi_floodEXP_half_.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 4,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            99.89921729566487,
                                            0.0,
                                            -0.00026949458523585647,
                                            7.877057231858849]},
                'crs': 'EPSG:4326'},
 'totalPatches': 12}
Running predictions...


  0%|          | 0/12 [00:00<?, ?it/s]

Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Writing patch 4...
Writing patch 5...
Writing patch 6...
Writing patch 7...
Writing patch 8...
Writing patch 9...
Writing patch 10...
Writing patch 11...
Started upload task with ID: W6GYEAALAPUGT5UXNLYELDYT
2016-12-15 2016-12-30
Running image export to Cloud Storage...
Image export completed.
Looking for TFRecord files...
['gs://geebucketwater/m2_TH_Cnn_L8SR_S1A_sl_CC_dp0.3/flood_half1/flood_thai_with_Multi_floodEXP_half_.tfrecord.gz']
gs://geebucketwater/m2_TH_Cnn_L8SR_S1A_sl_CC_dp0.3/flood_half1/flood_thai_with_Multi_floodEXP_half_.json
{'patchDimensions': [256, 256],
 'patchesPerRow': 4,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            99.89921729566487,
                                            0.0,
                                            -0.00026949458523585647,
    

  0%|          | 0/12 [00:00<?, ?it/s]

Writing predictions...
Writing patch 0...
Writing patch 1...
Writing patch 2...
Writing patch 3...
Writing patch 4...
Writing patch 5...
Writing patch 6...
Writing patch 7...
Writing patch 8...
Writing patch 9...
Writing patch 10...
Writing patch 11...
Started upload task with ID: GNQYAY4PXXHXGWMAODIFY3XD


## Objective 5: GlobalWaterNet - predicting in 10 locations

In [ ]:
configs_multi_global["L8SR_S1A_sl_CC_dp0.3"] = config.configuration("L8SR_S1A_sl_CC_dp0.3", BANDS1 = ["B2", "B3", "B4", "B5", "B6", "B7"], BANDS2=["VV", "VH", "angle", "slope"]\
                                                 , TRAIN_SIZE= TRAIN_SIZE, EVAL_SIZE = EVAL_SIZE\
                                                 , EPOCHS=10, BATCH_SIZE = 16, dropout_prob=0.3, LOSS=losses.get("categorical_crossentropy"), type_ = 2, country="TH")
conf = configs_multi_global["L8SR_S1A_sl_CC_dp0.3"]
preproc = preprocessing.Preprocessor(conf)
MODEL_DIR = 'gs://' + conf.BUCKET + "/" + conf.FOLDER + "/Models/" + conf.PROJECT_TITLE
model_custom = tf.keras.models.load_model(MODEL_DIR, custom_objects={'f1':metrics_.f1, "dice_p_cc": losses_.dice_p_cc})

image_bases = [th_image_base, tb_image_base, gm_image_base, brazil_image_base, mexico_image_base, \
               pakistan_image_base, egypt_image_base, cambodia_image_base, India_image_base, \
               Bangladesh_image_base]

images.doExport(th_image_base, kernel_buffer, th_region, conf)
images.doExport(tb_image_base, kernel_buffer, tb_region, conf)
images.doExport(gm_image_base, kernel_buffer, gm_region, conf)
images.doExport(brazil_image_base, kernel_buffer, brazil_region, conf)
images.doExport(mexico_image_base, kernel_buffer, mexico_region, conf)
images.doExport(pakistan_image_base, kernel_buffer, pakistan_region, conf)
images.doExport(egypt_image_base, kernel_buffer, egypt_region, conf)
images.doExport(cambodia_image_base, kernel_buffer, cambodia_region, conf)
images.doExport(India_image_base, kernel_buffer, India_region, conf)
images.doExport(Bangladesh_image_base, kernel_buffer, Bangladesh_region, conf)

for image_base in image_bases:
  out_image_asset, out_image_file, jsonFile = images.doPrediction_multiview_2(image_base, user_folder, \
                         kernel_buffer, model_custom, "bestglobal", conf)
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}